In [29]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16

def create_cnn_model(input_shape):
    base_model = VGG16(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False  # Freeze the base model layers

    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(emotion_labels), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model




# Define the CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),  # Use Input layer for input shape
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(emotion_labels), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load and preprocess the dataset
def load_data():
    # Assuming you have a dataset directory with subfolders for each emotion
    data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    
    train_data = data_gen.flow_from_directory('dataset/train',
                                              target_size=(48, 48),
                                              color_mode='grayscale',
                                              class_mode='categorical',
                                              batch_size=32,
                                              subset='training')
    
    val_data = data_gen.flow_from_directory('dataset/test',
                                            target_size=(48, 48),
                                            color_mode='grayscale',
                                            class_mode='categorical',
                                            batch_size=32,
                                            subset='validation')
    
    return train_data, val_data

# Train the model
train_data, val_data = load_data()
input_shape = (48, 48, 1)
model = create_cnn_model(input_shape)
model.fit(train_data, epochs=10, validation_data=val_data)

# Save the trained model
model.save('emotion_cnn_model.keras')


Found 22968 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.
Epoch 1/10
